# Conectar com Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

%cd /content/gdrive/MyDrive/Colab/jai2021-jupyter-colab/6.Jupyter.Avancado

!ls

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab/jai2021-jupyter-colab/6.Jupyter.Avancado
6.1.IPython.ipynb	 6.6.Widget.Simples.ipynb	 kernel2.svg
6.2.Superconjunto.ipynb  6.7.Widget.Personalizado.ipynb  kernel.svg
6.3.Magica.ipynb	 6.Exercicios.ipynb		 spotify.py
6.4.Uso.Magica.ipynb	 cell3.svg			 widgetcustom.svg
6.5.Exibicao.ipynb	 files


## Mágicas do IPython e Extensões

No notebook anterior foi apresentado como usar mágicas no IPython.

Este notebook tem os seguintes objetivos:
- Apresentar como criar novas mágicas
- Apresentar como exportá-las para extensões

Começamos o notebook carregando o arquivo com informações de artistas do Spotify para o Pandas.

Esse arquivo será usado para criar um "tocador" do Spotify para músicas de artistas dentro do notebook.


In [2]:
import pandas as pd
from IPython.core.magic import Magics, magics_class, line_magic
dfa = pd.read_csv("../dataset/spotify_artists_info_complete.tsv", sep="\t")
dfa.loc[0]

artist_id                                4gzpq5DPGxSnKTe4SA8HAU
name                                                   Coldplay
followers                                              29397183
popularity                                                   90
genres                                ['permanent wave', 'pop']
image_url     https://i.scdn.co/image/4ffd6710617d289699cc0d...
Name: 0, dtype: object

Perceba nesse resultado que o **Coldplay** possui `artist_id` **4gzpq5DPGxSnKTe4SA8HAU**.

Este `artist_id` é usado no iframe que o Spotify disponibiliza para exibir tocadores de artistas e músicas em sites.

```html
<iframe src="https://open.spotify.com/embed/{type}/{id}"
        width="360" height="180"
        frameborder="0" allowtransparency="true" 
        allow="encrypted-media"></iframe>
```

Para artista, `type` deve ser `artist`

Para música, `type` deve ser `track`
     

Usando a mágica `%%html` apresentada anteriormente, podemos mostrar o tocador do Coldplay:

In [3]:
%%html
<iframe src="https://open.spotify.com/embed/artist/4gzpq5DPGxSnKTe4SA8HAU"
        width="360" height="180"
        frameborder="0" allowtransparency="true" 
        allow="encrypted-media"></iframe>

Escrever assim funciona, mas repetir o código do iframe para cada artista pode ser um tanto complicado, então vamos facilitar com uma função:

In [4]:
EMBED_URL = (
    '<iframe src="https://open.spotify.com/embed/{type}/{id}"'
    ' width="{width}" height="{height}"'
    ' frameborder="0" allowtransparency="true"'
    ' allow="encrypted-media"></iframe>'
)

def display_artist(kernel, aid, width=360, height=180):
    return kernel.run_cell_magic('html', '', EMBED_URL.format(
        type="artist", id=aid,
        width=width, height=height
    ))

In [5]:
display_artist(get_ipython(), "4gzpq5DPGxSnKTe4SA8HAU")

### Mágica de linha

Essa forma de escrita funciona e usa apenas a sintaxe do Python, mas pode ser interessante ter isso como uma mágica do IPython.

In [6]:
from IPython.core.magic import Magics, magics_class, line_magic

@magics_class
class SpotifyMagics(Magics):
    
    @line_magic
    def artist(self, line):
        return display_artist(self.shell, line)
    
get_ipython().register_magics(SpotifyMagics)

Pontos importantes a se observar:

- `self.shell` já dá acesso ao kernel do IPython, dispensando o uso de `get_ipython()` na mágica
- É necessário criar uma classe de mágicas e registrar essas mágicas no kernel através do método `register_magics`

Com esse código, passa a ser possível usar uma mágica para visualizar o artista:

In [7]:
%artist 4gzpq5DPGxSnKTe4SA8HAU

### Argumentos

Apesar de funcionar desta forma, poderia ser possível passar `-w 400 -h 250` para alterar o tamanho do tocador.

Para isso usamos decoradores baseados no `argparse` do Python.

In [8]:
from IPython.core.magic import Magics, magics_class, line_magic
from IPython.core.magic_arguments import parse_argstring, magic_arguments, argument

@magics_class
class SpotifyMagics(Magics):
    
    @magic_arguments()
    @argument("artist_id", help="Id de artista")
    @argument("-w", "--width", type=int, default=360, help="Largura")
    @argument("-h", "--height", type=int, default=180, help="Altura")
    @line_magic
    def artist(self, line):
        args = parse_argstring(self.artist, line)
        return display_artist(
            self.shell, args.artist_id,
            args.width, args.height
        )
    
get_ipython().register_magics(SpotifyMagics)

O uso dos decoradores `@magic_arguments`, `@argument` e da função `parse_argstring` permite extrair argumentos do parâmetro linha.

Com isso, definimos uma mágica que se parece com linhas de comando e expressões bang:

In [9]:
%artist 4gzpq5DPGxSnKTe4SA8HAU -w 400 -h 240

Com os argumentos definidos desta forma, é possível acessar a documentação da mágica:

In [10]:
%artist?

### Mágica de célula

Outra alteração que podemos fazer na mágica anterior, é permitir que ela atue tanto como mágica de linha quanto como mágica de célula. 

Quando ela for uma mágica de célula, ela deve poder apresentar o tocador de vários artistas.

In [11]:
from IPython.display import HTML
from IPython.core.magic import line_cell_magic

@magics_class
class SpotifyMagics(Magics):
    
    @magic_arguments()
    @argument("ids", nargs="*", help="Ids de artistas")
    @argument("-w", "--width", type=int, default=360, help="Largura")
    @argument("-h", "--height", type=int, default=180, help="Altura")
    @line_cell_magic
    def artist(self, line, cell=""):
        args = parse_argstring(self.artist, line)
        ids = args.ids or cell.split('\n')
        result = []
        for aid in ids:
            if aid:
                result.append(EMBED_URL.format(
                    type="artist", id=aid,
                    width=args.width, height=args.height
                ))
        return HTML("<br>".join(result))
        
get_ipython().register_magics(SpotifyMagics)

Principais mudanças:

- `@line_magic` -> `@line_cell_magic`
- `artist(self, line)` -> `artist(self, line, cell="")`
- `nargs="*"` no argumento ids para aceitar vários
- Construção de lista de resultados com vários IFrames
- Resultado final usando classe HTML ao invés da mágica de célula `%%html`. A razão disso é antecipar uma mudança que será necessária no futuro

Para testar, vamos selecionar os 3 primeiros artistas:

In [12]:
dfa.loc[0:2].artist_id

0    4gzpq5DPGxSnKTe4SA8HAU
1    6Te49r3A6f5BiIgBRxH7FH
2    4QrBoWLm2WNlPdbFhmlaUZ
Name: artist_id, dtype: object

A partir disso, podemos usar na mágica:

In [13]:
%%artist -h 155
4gzpq5DPGxSnKTe4SA8HAU
6Te49r3A6f5BiIgBRxH7FH
4QrBoWLm2WNlPdbFhmlaUZ

### Mágica para música

Vamos criar também uma mágica `%track` para a visualização de tocadores de música.

In [14]:
from IPython.display import HTML
from IPython.core.magic import line_cell_magic

@magics_class
class SpotifyMagics(Magics):
    
    def embed_player(self, fn, line, cell, type_):
        args = parse_argstring(fn, line)
        ids = args.ids or cell.split('\n')
        result = []
        for aid in ids:
            if aid:
                result.append(EMBED_URL.format(
                    type=type_, id=aid,
                    width=args.width, height=args.height
                ))
        return HTML("<br>".join(result))
    
    @magic_arguments()
    @argument("ids", nargs="*", help="Ids de artistas")
    @argument("-w", "--width", type=int, default=360, help="Largura")
    @argument("-h", "--height", type=int, default=180, help="Altura")
    @line_cell_magic
    def artist(self, line, cell=""):
        return self.embed_player(self.artist, line, cell, 'artist')
        
    @magic_arguments()
    @argument("ids", nargs="*", help="Ids de músicas")
    @argument("-w", "--width", type=int, default=300, help="Largura")
    @argument("-h", "--height", type=int, default=80, help="Altura")
    @line_cell_magic
    def track(self, line, cell=""):
        return self.embed_player(self.track, line, cell, 'track')

get_ipython().register_magics(SpotifyMagics)

### Extensão IPython

Por enquanto nossa mágica é valida para este único notebook, mas seria bom podermos usar em outros notebooks.

Uma forma de atingir este objetivo é através da criação de uma extensão para o IPython.

Extensões são arquivos Python comuns que podem ser carregados através da mágica `%load_ext`. 

Esta mágica importa a extensão e executa a função `load_ipython_extension` passando o kernel como parâmetro.

In [15]:
%%writefile spotify.py
from IPython.core.magic import Magics, magics_class, line_magic, line_cell_magic
from IPython.core.magic_arguments import parse_argstring, magic_arguments, argument
from IPython.display import HTML

EMBED_URL = (
    '<iframe src="https://open.spotify.com/embed/{type}/{id}"'
    ' width="{width}" height="{height}"'
    ' frameborder="0" allowtransparency="true"'
    ' allow="encrypted-media"></iframe>'
)

# Define classe de mágicas
@magics_class
class SpotifyMagics(Magics):
    
    def embed_player(self, fn, line, cell, type_):
        args = parse_argstring(fn, line)
        ids = args.ids or cell.split('\n')
        result = []
        for aid in ids:
            if aid:
                result.append(EMBED_URL.format(
                    type=type_, id=aid,
                    width=args.width, height=args.height
                ))
        return HTML("<br>".join(result))
    
    @magic_arguments()
    @argument("ids", nargs="*", help="Ids de artistas")
    @argument("-w", "--width", type=int, default=360, help="Largura")
    @argument("-h", "--height", type=int, default=180, help="Altura")
    @line_cell_magic
    def artist(self, line, cell=""):
        return self.embed_player(self.artist, line, cell, 'artist')
        
    @magic_arguments()
    @argument("ids", nargs="*", help="Ids de músicas")
    @argument("-w", "--width", type=int, default=300, help="Largura")
    @argument("-h", "--height", type=int, default=80, help="Altura")
    @line_cell_magic
    def track(self, line, cell=""):
        return self.embed_player(self.track, line, cell, 'track')
        
def load_ipython_extension(kernel):
    # Registra mágicas
    kernel.register_magics(SpotifyMagics)


Overwriting spotify.py


## Conclusão

Este notebook apresentou a definição de uma mágica e de uma extensão para o IPython.

O próximo notebook ([6.4.Uso.Magica.ipynb](https://drive.google.com/file/d/1hgKwLckTKnxEcctUDQF_4YWjctPHBhbi/view?usp=sharing)) apresenta o uso da extensão.